This function computes trajectories of an autonomous equation starting at $ \mathbf{x}_0 $ given an Interpolant $ \ $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| s_dummy | array (Ns, ) | dummy time-interval of length Ns |
| x | array (2, ) | $ \mathbf{x}_0 $ |
| Interpolant | list (2,) | Interpolant[0]: Interpolant for x-component of $ \mathbf{x}' $ <br /> Interpolant[1]: Interpolant for y-component of $ \mathbf{x}' $ |

In [1]:
# import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

# Import numpy 
import numpy as np

# Import pi from math tools
from math import pi

# import time counter
import time

In [1]:
def AutonomousODE(s_dummy, x0, Interpolant):
    
    Interpolant_x = Interpolant[0]
    Interpolant_y = Interpolant[1]
    
    ds = s_dummy[1] - s_dummy[0]
    
    x = np.zeros((2, len(s_dummy)))
    xprime = np.zeros((2, len(s_dummy)))
    
    x0 = x0%(2*pi)
    
    for i in range(len(s_dummy)):
        x[:, i] = x0
        x0, xprime[:,i] = RK4_integration(x0, ds, Interpolant_x, Interpolant_y)
    
    return x, xprime

In [2]:
def RK4_integration(x0, ds, Interpolant_x, Interpolant_y):
    
    # Define starting point.
    x1x = x0[0]
    x1y = x0[1]
    
    # compute derivative
    k1x = ds * Interpolant_x(x1y, x1x)[0][0]
    k1y = ds * Interpolant_y(x1y, x1x)[0][0]
    
    #  position and time at the first midpoint.
    x2x = (x1x + .5 * k1x)%(2*pi)
    x2y = (x1y + .5 * k1y)%(2*pi)
    
    # compute derivative
    k2x = ds * Interpolant_x(x2y, x2x)[0][0]
    k2y = ds * Interpolant_y(x2y, x2x)[0][0]

    # Update position at the second midpoint.
    x3x = (x1x + .5 * k2x)%(2*pi)
    x3y = (x1y + .5 * k2y)%(2*pi)
    
    # compute derivative
    k3x = ds * Interpolant_x(x3y, x3x)[0][0]
    k3y = ds * Interpolant_y(x3y, x3x)[0][0]
    
    # Update position at the endpoint.
    x4x = (x1x + k3x)%(2*pi)
    x4y = (x1y + k3y)%(2*pi)
    
    # compute derivative
    k4x = ds * Interpolant_x(x4y, x4x)[0][0]
    k4y = ds * Interpolant_y(x4y, x4x)[0][0]
    
    # compute derivatives and updated position of particle after 1 integration step

    x_prime_update_x = 1 / 6 * (k1x + 2 * k2x + 2 * k3x + k4x)
    x_prime_update_y = 1 / 6 * (k1y + 2 * k2y + 2 * k3y + k4y)
    
    x_prime_update = np.array([x_prime_update_x, x_prime_update_y])
    
    x_update = np.array([x1x+x_prime_update_x, x1y+x_prime_update_y])%(2*pi)

    return x_update, x_prime_update